In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

In [2]:
# read function
def readCsv(name):
    try:
        return pd.read_csv(name)
    except UnicodeDecodeError:
        try:
            return pd.read_csv(name, encoding='latin1')
        except UnicodeDecodeError:
            return pd.read_csv(name, encoding='ISO-8859-1')

In [3]:
apptsDf = readCsv('csv/appointments.csv')
apptsDf.drop_duplicates(subset='apptid', inplace=True)

In [4]:
clinicsDf = readCsv('csv/clinics.csv')
clinicsDf.drop_duplicates(subset='clinicid', inplace=True)

In [5]:
doctorsDf = readCsv('csv/doctors.csv')
doctorsDf.drop_duplicates(subset='doctorid', inplace=True)

In [6]:
pxDf = readCsv('csv/px.csv')
pxDf.drop_duplicates(subset='pxid', inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15584\3144380163.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(name)


In [7]:
apptsDf = apptsDf[apptsDf["clinicid"].isin(clinicsDf["clinicid"])]
apptsDf = apptsDf[apptsDf["doctorid"].isin(doctorsDf["doctorid"])]
apptsDf = apptsDf[apptsDf["pxid"].isin(pxDf["pxid"])]

In [8]:
clinicsDf = clinicsDf[clinicsDf["clinicid"].isin(apptsDf["clinicid"])]
doctorsDf = doctorsDf[doctorsDf["doctorid"].isin(apptsDf["doctorid"])]
pxDf = pxDf[pxDf["pxid"].isin(apptsDf["pxid"])]

In [9]:
pxDf = pxDf.rename(columns={'age': 'pxage', 'gender': 'pxgender'})
doctorsDf = doctorsDf.rename(columns={'age': 'DoctorAge', 'mainspecialty': 'MainSpecialty'})

In [10]:
combinedDf = apptsDf.merge(clinicsDf, on='clinicid')
combinedDf = combinedDf.merge(doctorsDf, on='doctorid')
combinedDf = combinedDf.merge(pxDf, on='pxid')

In [11]:
combinedDf.drop(['pxid', 'clinicid', 'doctorid'], axis=1, inplace=True)

In [12]:
combinedDf.drop(['TimeQueued', 'QueueDate','EndTime','pxage','pxgender','hospitalname'], axis=1, inplace=True)

In [13]:
combinedDf['StartTime'] = pd.to_datetime(combinedDf['StartTime'])
combinedDf['StartTime'] = combinedDf['StartTime'].dt.hour

In [14]:
combinedDf = combinedDf.rename(columns={'StartTime': 'StartHour'})

In [15]:
combinedDf.columns

Index(['apptid', 'status', 'StartHour', 'type', 'Virtual', 'IsHospital',
       'City', 'Province', 'RegionName', 'MainSpecialty', 'DoctorAge'],
      dtype='object')

In [16]:
Luzon = 'National Capital Region (NCR)', 'CALABARZON (IV-A)', 'Ilocos Region (I)', 'Bicol Region (V)','Central Luzon (III)'
luzonDf = combinedDf[combinedDf['RegionName'].isin(Luzon)]
visMinDf = combinedDf[~combinedDf['RegionName'].isin(Luzon)]

In [17]:
def table(cnx):
    cursor = cnx.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS distributed_database")
    cursor.execute("USE distributed_database")
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS appointments (
            `apptid` VARCHAR(50) NOT NULL,
            `status` VARCHAR(50) NULL,
            `StartHour` int NULL,
            `type` VARCHAR(45) NULL,
            `Virtual` BOOL NULL,
            `IsHospital` BOOL NULL,
            `City` VARCHAR(255) NULL,
            `Province` VARCHAR(255) NULL,
            `RegionName` VARCHAR(255) NULL,
            `MainSpecialty` VARCHAR(255) NULL,
            `DoctorAge` INT NULL,
            PRIMARY KEY (`apptid`)
        )
    """)
    cnx.commit()


In [18]:
def logTable(cnx):
    cursor = cnx.cursor()
    cursor.execute("USE distributed_database")
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS appointment_log (
            `apptid` VARCHAR(50) NOT NULL,
            `status` VARCHAR(50) NULL,
            `StartHour` int NULL,
            `type` VARCHAR(45) NULL,
            `Virtual` BOOL NULL,
            `IsHospital` BOOL NULL,
            `City` VARCHAR(255) NULL,
            `Province` VARCHAR(255) NULL,
            `RegionName` VARCHAR(255) NULL,
            `MainSpecialty` VARCHAR(255) NULL,
            `DoctorAge` INT NULL,
            `Query` ENUM("INSERT", "DELETE", "UPDATE", "SENTINEL") NOT NULL,
            `Node` ENUM('0','1', '2', "SENTINEL") NOT NULL,
            `ID` INT NOT NULL,
            PRIMARY KEY (`ID`)
            );
    """)
    cursor.execute("""
        INSERT INTO appointment_log (`apptid`, `status`, `StartHour`, `type`, `Virtual`, `IsHospital`, `City`, `Province`, `RegionName`, `MainSpecialty`, `DoctorAge`, `Query`, `Node`, `ID`)
        VALUES ('SENTINEL', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'SENTINEL', 'SENTINEL',0);
    """)
    cnx.commit()

In [19]:
cnx = mysql.connector.connect(
    host='ccscloud.dlsu.edu.ph',
    port=20141,
    user='root',
)
table(cnx)
logTable(cnx)
cnx.close()

In [20]:
engine = create_engine('mysql+pymysql://root@ccscloud.dlsu.edu.ph:20141/distributed_database')
combinedDf.to_sql('appointments', con=engine, if_exists='replace', index=False)
engine.dispose()

In [21]:
cnx = mysql.connector.connect(
    host='ccscloud.dlsu.edu.ph',
    port=20142,
    user='root',
)
table(cnx)
logTable(cnx)
cnx.close()

In [22]:
engine = create_engine('mysql+pymysql://root@ccscloud.dlsu.edu.ph:20142/distributed_database')
luzonDf.to_sql('appointments', con=engine, if_exists='replace', index=False)
engine.dispose()


In [23]:
cnx = mysql.connector.connect(
    host='ccscloud.dlsu.edu.ph',
    port=20143,
    user='root',
)
table(cnx)
logTable(cnx)
cnx.close()

In [24]:
engine = create_engine('mysql+pymysql://root@ccscloud.dlsu.edu.ph:20143/distributed_database')
visMinDf.to_sql('appointments', con=engine, if_exists='replace', index=False)
engine.dispose()